In [ ]:
import os
import pickle

# Set your directory path (adjust based on your dataset folder name)
pkl_dir = "/kaggle/input/households"

# List all .pkl files
pkl_files = [f for f in os.listdir(pkl_dir) if f.endswith('.pkl')]

# Load all pkl files into a list or dict
data_list = []

for file in pkl_files:
    file_path = os.path.join(pkl_dir, file)
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        data_list.append(data)

# Optional: Load into a dictionary using filename as key
# data_dict = {file: pickle.load(open(os.path.join(pkl_dir, file), 'rb')) for file in pkl_files}


In [ ]:
import pandas as pd

# Combine into one big DataFrame
combined_df = pd.concat(data_list, ignore_index=True)


In [ ]:
print(combined_df.shape)
print(combined_df.dtypes)
combined_df.head()


In [ ]:
from datetime import datetime

# 6. Define date windows
train_end = '2016-04-24'
valid_start = '2016-04-10'     # <-- must be ≤ 2016-04-24
valid_end = '2016-04-24'
sequence_length = 28

# First, convert 'date' to datetime if not already
combined_df['date'] = pd.to_datetime(combined_df['date'])

# Define the earliest date needed to cover the sequence length
min_date = pd.to_datetime(valid_start) - pd.Timedelta(days=sequence_length)

# Filter the dataset to include only the date range needed
filtered_df = combined_df[
    (combined_df['date'] >= min_date) &
    (combined_df['date'] <= pd.to_datetime(train_end))
].copy()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder


# ✅ 2. Make sure date is datetime (you already did this earlier, but safe to ensure)
filtered_df['date'] = pd.to_datetime(filtered_df['date'])

# ✅ 3. Define feature columns
categorical_cols = ['event_type', 'is_weekend', 'snap_flag', 'day_of_week', 'month', 'year', 'state_id', 'store_id']
numeric_cols = [
    'sales_mean_7', 'sales_mean_28', 'sales_max_7', 'sales_max_28',
    'sales_min_7', 'sales_min_28', 'sales_std_7', 'sales_std_28',
    'price_mean_7', 'price_mean_28', 'price_max_7', 'price_max_28',
    'price_min_7', 'price_min_28', 'price_std_7', 'price_std_28',
    'sales', 'sell_price'
]

# ✅ 4. Label encode categorical columns
for col in categorical_cols:
    filtered_df[col] = LabelEncoder().fit_transform(filtered_df[col].astype(str))

# ✅ 5. Convert numeric columns safely
for col in numeric_cols:
    filtered_df[col] = pd.to_numeric(filtered_df[col], errors='coerce')

# ✅ 6. Normalize features
feature_cols = categorical_cols + numeric_cols
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(filtered_df[feature_cols])

# ✅ 7. Final scaled DataFrame
scaled_df = pd.DataFrame(scaled_values, columns=feature_cols)
scaled_df['sales'] = filtered_df['sales'].values  # optional, but often useful
scaled_df['date'] = filtered_df['date'].values
scaled_df['d'] = filtered_df['d'].values


In [ ]:
# 7. Slice train and validation sets
# ✅ Include 28-day buffer before valid_start
train_df = scaled_df[
    (scaled_df['date'] >= '2015-04-25') & (scaled_df['date'] <= train_end)
].reset_index(drop=True)

valid_df = scaled_df[
    (scaled_df['date'] >= '2016-03-28') & (scaled_df['date'] <= '2016-05-22')
].reset_index(drop=True)

print("Train shape:", train_df.shape)
print("Valid shape:", valid_df.shape)

train_df = train_df.dropna().reset_index(drop=True)
print("✅ Train shape (after dropna):", train_df.shape)

# 8. Memory-efficient training sequence generator
def sequence_generator_multistep(df, input_len=28, output_len=28, batch_size=32, target_col='sales'):
    feature_cols = [col for col in df.columns if col not in ['date', 'd']]
    target_idx = feature_cols.index(target_col)
    data_array = df[feature_cols].values.astype('float32')
    i = input_len
    while True:
        X_batch, y_batch = [], []
        for _ in range(batch_size):
            if i + output_len >= len(data_array):
                i = input_len  # restart
            X_seq = data_array[i-input_len:i]
            y_seq = data_array[i:i+output_len, target_idx]  # grab 28 days future
            X_batch.append(X_seq)
            y_batch.append(y_seq)
            i += 1
        yield np.array(X_batch), np.log1p(np.array(y_batch))  # log transform y

# 9. Fast validation sequence creator
def create_multistep_sequences(df, input_len=28, output_len=28, target_col='sales', min_target_date=None):
    feature_cols = [col for col in df.columns if col not in ['date', 'd']]
    target_idx = feature_cols.index(target_col)

    data_array = df[feature_cols].values.astype('float32')
    date_array = df['date'].values

    X, y = [], []
    for i in range(input_len, len(data_array) - output_len):
        if min_target_date is not None and date_array[i] < min_target_date:
            continue
        X_seq = data_array[i-input_len:i]
        y_seq = data_array[i:i+output_len, target_idx]
        X.append(X_seq)
        y.append(y_seq)
    return np.array(X, dtype='float32'), np.log1p(np.array(y, dtype='float32'))


# 10. Create validation set
X_valid, y_valid = create_multistep_sequences(
    valid_df,
    input_len=28,
    output_len=28,
    min_target_date=pd.to_datetime(valid_start)
)

print(X_valid.shape)  # (num_samples, 28, num_features)
print(y_valid.shape)  # (num_samples, 28)

# ✅ 建立一小段 training set for memory-safe training generator
# 你不需要事先創建 X_train / y_train，全靠 generator 即可
# 但 y_valid 要先轉換：
y_valid = np.log1p(y_valid)  # ✅ 避免 loss 爆炸


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import numpy as np

# ✅ 1. Generator 改為多步輸出
def sequence_generator_multistep(df, input_len=28, output_len=28, batch_size=32, target_col='sales'):
    feature_cols = [col for col in df.columns if col not in ['date', 'd']]
    target_idx = feature_cols.index(target_col)
    data_array = df[feature_cols].values.astype('float32')
    i = input_len
    while True:
        X_batch, y_batch = [], []
        for _ in range(batch_size):
            if i + output_len >= len(data_array):
                i = input_len
            X_seq = data_array[i-input_len:i]
            y_seq = data_array[i:i+output_len, target_idx]
            if np.isnan(X_seq).any() or np.isnan(y_seq).any():
                print(f"🚨 NaN at step {_}")
            X_batch.append(X_seq)
            y_batch.append(y_seq)
            i += 1
        yield np.array(X_batch), np.log1p(np.array(y_batch))  # log1p 避免 loss 爆炸

# ✅ 2. 模型輸出 28 維（你預測 28 天）
n_features = 26  # 記得根據你的 input features 調整
model = Sequential([
    GRU(64, input_shape=(28, n_features), return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(28)  # 改成 28-day forecast
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_squared_error')

# ✅ 3. Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint_cb = ModelCheckpoint("best_gru_model.keras", monitor="val_loss", save_best_only=True)

# ✅ 4. 訓練設定
batch_size = 16
train_gen = sequence_generator_multistep(train_df, 28, 28, batch_size=batch_size)
num_samples = len(train_df) - 28 - 28
steps_per_epoch = num_samples // batch_size


# ✅ 5. 不要重複 log1p，直接用 create_sequences_fast return 的結果
history = model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stop, checkpoint_cb],
    verbose=1
)

# ✅ Save the model manually (so it can be loaded later)
model.save("best_gru_model.keras") 

# ✅ 6. 評估（記得還原 expm1）
y_pred = model.predict(X_valid)
y_pred_restored = np.expm1(y_pred)
y_valid_restored = np.expm1(y_valid)

rmse = np.sqrt(mean_squared_error(y_valid_restored.flatten(), y_pred_restored.flatten()))
print(f"✅ Final Validation RMSE: {rmse:.4f}")


Testing Evaluation Data in below

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# ✅ 1. Load and parse evaluation batch
eval_df = pd.read_pickle("/kaggle/input/hobbby-eval/hobbies_batch_eval_0.pkl 2")
eval_df['date'] = pd.to_datetime(eval_df['date'])

# ✅ 2. Date filtering (match training logic)
valid_start = '2016-04-10'
train_end = '2016-04-24'
sequence_length = 28
min_date = pd.to_datetime(valid_start) - pd.Timedelta(days=sequence_length)
eval_df = eval_df[(eval_df['date'] >= min_date) & (eval_df['date'] <= '2016-05-22')].copy()

# ✅ 3. Combine event_type_1 and event_type_2 into 'event_type'
eval_df['event_type'] = (
    eval_df['event_type_1'].fillna('').astype(str) + '_' +
    eval_df['event_type_2'].fillna('').astype(str)
).str.strip('_')

# ✅ 4. Define columns
categorical_cols = [
    'event_type', 'is_weekend', 'snap_flag', 'month', 'year',
    'state_id', 'store_id'
]
numeric_cols = [
    'sales_mean_7', 'sales_mean_28', 'sales_max_7', 'sales_max_28',
    'sales_min_7', 'sales_min_28', 'sales_std_7', 'sales_std_28',
    'price_mean_7', 'price_mean_28', 'price_max_7', 'price_max_28',
    'price_min_7', 'price_min_28', 'price_std_7', 'price_std_28',
    'sales'  # target column still kept
]
feature_cols = categorical_cols + numeric_cols

# ✅ 5. Label encode categoricals
for col in categorical_cols:
    eval_df[col] = LabelEncoder().fit_transform(eval_df[col].astype(str))

# ✅ 6. Convert numerics safely
for col in numeric_cols:
    eval_df[col] = pd.to_numeric(eval_df[col], errors='coerce')

# ✅ 7. Scale features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(eval_df[feature_cols])

# ✅ 8. Final DataFrame
scaled_df = pd.DataFrame(scaled_features, columns=feature_cols)
scaled_df['sales'] = eval_df['sales'].values  # keep original target
scaled_df['date'] = eval_df['date'].values
scaled_df['id'] = eval_df['id'].values

print("✅ Scaled eval_df is ready for GRU inference")


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# ✅ Load model
model = load_model("best_gru_model.keras")

# ✅ Step 1: Get expected features from training data
train_feature_cols = [col for col in train_df.columns if col not in ['sales', 'date', 'id']]
expected_feature_count = len(train_feature_cols)
print("✅ Model expects features:", expected_feature_count)

# ✅ Step 2: Align eval features
eval_feature_cols = list(scaled_df.columns)
missing_cols = list(set(train_feature_cols) - set(eval_feature_cols))
print("🚨 Missing columns in eval:", missing_cols)

# Add missing columns with zeros
for col in missing_cols:
    scaled_df[col] = 0.0

# Reorder columns to match training feature order
scaled_df = scaled_df[train_feature_cols + ['sales', 'date', 'id']]


In [ ]:
# ✅ Step 3: Make predictions
all_preds = []

for item_id, group in scaled_df.groupby("id"):
    group = group.sort_values("date")
    if len(group) < 28:
        continue

    X_input = group[train_feature_cols].values[-28:].astype("float32")
    X_input = np.expand_dims(X_input, axis=0)

    y_pred_log = model.predict(X_input, verbose=0)
    y_pred = np.expm1(y_pred_log).flatten()

    row = {"id": item_id}
    row.update({f"F{i+1}": y for i, y in enumerate(y_pred)})
    all_preds.append(row)

In [ ]:
# ✅ Step 4: Save result
submission_df = pd.DataFrame(all_preds)
submission_df.to_csv("hobby_forecast.csv", index=False)
print("✅ Saved hobby_forecast.csv with shape:", submission_df.shape)

don't use validation data

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# ✅ Load trained GRU model
model = load_model("best_gru_model.keras")
expected_feature_count = model.input_shape[-1]  # e.g., 26
print(f"✅ Model expects input shape: (None, 28, {expected_feature_count})")

# ✅ Step 1: Identify features used in training
train_feature_cols = [col for col in train_df.columns if col not in ['sales', 'date', 'id']]

# Remove any leftover categorical columns that are not numeric
non_numeric_cols = train_df[train_feature_cols].select_dtypes(include=['object']).columns.tolist()
if non_numeric_cols:
    print(f"🚨 Dropping non-numeric columns: {non_numeric_cols}")
    train_feature_cols = [col for col in train_feature_cols if col not in non_numeric_cols]


# ✅ Step 2: Add dummy features if needed
current_feature_count = len(train_feature_cols)
if current_feature_count < expected_feature_count:
    pad_count = expected_feature_count - current_feature_count
    for i in range(pad_count):
        pad_col = f"pad_{current_feature_count + i}"
        train_df[pad_col] = 0.0
        train_feature_cols.append(pad_col)
    print(f"⚠️ Padded {pad_count} column(s) to match model input")

X_input = train_df[train_feature_cols].values[-28:].astype("float32")
X_input = np.expand_dims(X_input, axis=0)  # shape: (1, 28, n_features)
y_pred_log = model.predict(X_input, verbose=0)
y_pred = np.expm1(y_pred_log).flatten()

# ✅ Step 5: Build submission DataFrame
submission_df = pd.DataFrame({
    "id": [f"model_{i+1}" for i in range(1)],
    **{f"F{i+1}": [y] for i, y in enumerate(y_pred)}
})

# ✅ Save to CSV
submission_df.to_csv("submission_forecast_hobby.csv", index=False)
print("📦 Saved submission_forecast_hobby.csv")


No validation data

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# ✅ Load model
model = load_model("best_gru_model.keras")
expected_feature_count = model.input_shape[-1]
print(f"✅ Model expects input shape: (None, 28, {expected_feature_count})")

# ✅ Prepare features
train_feature_cols = [col for col in train_df.columns if col not in ['sales', 'date', 'd', 'id']]
non_numeric_cols = train_df[train_feature_cols].select_dtypes(include=['object']).columns.tolist()
if non_numeric_cols:
    print(f"🚨 Dropping non-numeric columns: {non_numeric_cols}")
    train_feature_cols = [col for col in train_feature_cols if col not in non_numeric_cols]

# ✅ Pad dummy features if needed
current_feature_count = len(train_feature_cols)
if current_feature_count < expected_feature_count:
    pad_count = expected_feature_count - current_feature_count
    for i in range(pad_count):
        pad_col = f"pad_{current_feature_count + i}"
        train_df[pad_col] = 0.0
        train_feature_cols.append(pad_col)
    print(f"⚠️ Padded with {pad_count} columns")

# ✅ Step 1: Predict F1–F28
X_input = train_df[train_feature_cols].values[-28:].astype("float32")
X_input = np.expand_dims(X_input, axis=0)  # shape: (1, 28, n_features)
y_pred_1 = np.expm1(model.predict(X_input, verbose=0)).flatten()

# ✅ Step 2: Use F1–F28 as next input for F29–F56
future_df = train_df.iloc[-28:].copy()
future_df['sales'] = y_pred_1
X_input_2 = future_df[train_feature_cols].values.astype("float32")
X_input_2 = np.expand_dims(X_input_2, axis=0)
y_pred_2 = np.expm1(model.predict(X_input_2, verbose=0)).flatten()

# ✅ Step 3: Create 1-row submission
y_full = np.concatenate([y_pred_1, y_pred_2])
submission_df = pd.DataFrame({
    "id": ["model_1"],
    **{f"F{i+1}": [val] for i, val in enumerate(y_full)}
})

# ✅ Match to actual Kaggle sample_submission.csv
sample = pd.read_csv("/kaggle/input/sub-v2/submission.csv")
final_submission = pd.concat([sample.drop(columns=sample.columns[1:]), submission_df.iloc[0:1].drop(columns=['id'])], axis=1)
final_submission["id"] = sample["id"]
final_submission = final_submission[sample.columns]  # keep order

# ✅ Save final
final_submission.to_csv("submission_final_hobby.csv", index=False)
print("🎯 Saved submission_final_hobby.csv with shape:", final_submission.shape)


In [ ]:
import shutil

# 將 CSV 壓縮成 zip（Kaggle 支援下載 zip，比較穩）
shutil.make_archive("/kaggle/working/submission_final_hobby", 'zip', "/kaggle/working", "submission_final_hobby.csv")
